# ProbSpace: YouTube動画視聴回数予測

In [ ]:
SKF = False

LGB = True
XGB = True
CAT = True

DATASET = 1

out_dir = "out_tmp"
!mkdir -p $out_dir

In [ ]:
import pandas as pd
import numpy as np
import scipy

import itertools
import os, datetime, gc, glob, re, random
import time, datetime
import pickle
from tqdm.notebook import tqdm
from imblearn.over_sampling import SMOTE
import optuna
import bhtsne, umap

from janome.tokenizer import Tokenizer
from janome.analyzer import Analyzer
from janome.tokenfilter import *
from janome.charfilter import UnicodeNormalizeCharFilter, RegexReplaceCharFilter
import unicodedata

import lightgbm as lgb
import xgboost as xgb
from catboost import Pool, CatBoostRegressor, CatBoostClassifier

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.linear_model import BayesianRidge, ElasticNet, Lasso, LogisticRegression, Ridge, SGDRegressor
from sklearn.ensemble import AdaBoostRegressor, BaggingRegressor
from sklearn.ensemble import StackingRegressor, VotingRegressor
from sklearn.ensemble import ExtraTreesRegressor, GradientBoostingRegressor, RandomForestRegressor

from ngboost import NGBRegressor
from ngboost.ngboost import NGBoost
from ngboost.learners import default_tree_learner
from ngboost.scores import MLE, CRPS, LogScore
from ngboost.distns import Normal, LogNormal

from sklearn.linear_model import BayesianRidge, ElasticNet, Lasso, LogisticRegression, Ridge, SGDRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.cluster import KMeans, MiniBatchKMeans, DBSCAN
from sklearn.model_selection import KFold, RepeatedKFold, StratifiedKFold, ShuffleSplit, GroupShuffleSplit, StratifiedShuffleSplit
from sklearn.model_selection import cross_validate, cross_val_predict, train_test_split


from sklearn.metrics import mean_squared_error, roc_auc_score, log_loss
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, Normalizer, RobustScaler, QuantileTransformer, PowerTransformer
from sklearn.feature_selection import SelectFromModel, RFE, SelectPercentile, SelectKBest

import tensorflow as tf
import tensorflow_addons as tfa

from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import backend as K
from tensorflow.keras import utils
from tensorflow.keras.optimizers import Adadelta, Adagrad, Adam, Adamax, Ftrl, Nadam, RMSprop, SGD
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping, TensorBoard, LambdaCallback

from tensorflow.keras import layers
from tensorflow.keras.layers import Concatenate, Lambda
from tensorflow.keras.layers import Activation, Average, Dense, Dropout, Flatten, BatchNormalization, LeakyReLU, Input
from tensorflow.keras.layers import Conv2D, SeparableConv2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D

import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno 

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 100)

In [ ]:
start = datetime.datetime.now()

In [ ]:
# Function for variable description
def description(df):
    summary = pd.DataFrame(df.dtypes, columns=['dtypes'])
    summary = summary.reset_index()
    summary['Name'] = summary['index']
    summary = summary[['Name','dtypes']]
    summary['Missing'] = df.isnull().sum().values    
    summary['Uniques'] = df.nunique().values
    summary['Mean'] = np.nanmean(df, axis=0).astype(df.dtypes)
    summary['Std'] = np.nanstd(df, axis=0).astype(df.dtypes)
    summary['Minimum'] = np.nanmin(df, axis=0).astype(df.dtypes)
    summary['Maximum'] = np.nanmax(df, axis=0).astype(df.dtypes)
    summary['First Value'] = df.iloc[0].values
    summary['Second Value'] = df.iloc[1].values
    summary['Third Value'] = df.iloc[2].values
    summary['dimension'] = str(df.shape)
    return summary

In [ ]:
def get_hist(target):
    plt.hist(target, bins=100)

    print("max:  {:>10,.6f}".format(target.max()))
    print("min:  {:>10,.6f}".format(target.min()))
    print("mean: {:>10,.6f}".format(target.mean()))
    print("std:  {:>10,.6f}".format(target.std()))
    
    return

def get_hist4(target1, title1, target2, title2, target3, title3, target4, title4):
    fig = plt.figure(figsize=(18, 18))

    ax1 = fig.add_subplot(5,1,1)
    ax2 = fig.add_subplot(5,1,2)
    ax3 = fig.add_subplot(5,1,3)
    ax4 = fig.add_subplot(5,1,4)
    ax5 = fig.add_subplot(5,1,5)

    ax1.set_title(title1)
    ax2.set_title(title2)
    ax3.set_title(title3)
    ax4.set_title(title4)
    ax5.set_title("OVERALL")
    
    ax1.hist(target1, bins=100)
    ax2.hist(target2, bins=100)
    ax3.hist(target3, bins=100)
    ax4.hist(target4, bins=100)

    ax5.hist(target1, bins=100, alpha=0.2, color='red')
    ax5.hist(target2, bins=100, alpha=0.2, color='green')
    ax5.hist(target3, bins=100, alpha=0.2, color='blue')
    #ax5.hist(target4, bins=100, alpha=0.2, color='grey')

    fig.show()

    return


## Load Data

In [ ]:
%%time

# for train/test data
train_data = pd.read_csv("./input/train_data.csv")
test_data = pd.read_csv("./input/test_data.csv")

# use predicted dislikes,likes,comment_out
dislikes_pred = pd.read_csv("./input/dislikes_pred_new.csv")
likes_pred = pd.read_csv("./input/likes_pred_new.csv")
comments_pred = pd.read_csv("./input/comment_count_pred_new.csv")

y = np.log1p(train_data['y']).copy()
y_bin = pd.cut(train_data['y'], [0, 10, 100,1000,10000,100000,1000000,10000000000], labels=[1,2,3,4,5,6,7])
y_bin = y_bin.astype(int)
test_id = test_data.id

train = train_data.drop(['id', 'y'], axis=1).copy()
test  = test_data.drop(['id'], axis=1).copy()

traintest = pd.concat([train, test]).reset_index(drop=True)


## 目的変数の分布

In [ ]:
get_hist(y)

## missing data

In [ ]:
for col in train_data.columns:
    print("{:<20}: {} ({:.2f}%)".format(col, train_data[col].isnull().sum(), train_data[col].isnull().sum()/train_data.shape[0]*100))

In [ ]:
for col in test_data.columns:
    print("{:<20}: {} ({:.2f}%)".format(col, test_data[col].isnull().sum(), test_data[col].isnull().sum()/test_data.shape[0]*100))

# seedの固定化

In [ ]:
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    #torch.manual_seed(seed)
    #torch.cuda.manual_seed(seed)
    #torch.backends.cudnn.deterministic = True

# 特徴量生成

In [ ]:
def create_features(df, features):
    # 欠損値処理
    df['tags'].fillna("[none]", inplace=True)
    df['description'].fillna(df['tags'].replace("|", " ") + df['title'], inplace=True)

    tokenizer = Tokenizer()
    char_filters = [UnicodeNormalizeCharFilter(), RegexReplaceCharFilter(r"[0123456789!#$%&()=~|\-^\\@`{;:+*},./\<>?_♪®」—]", "")]
    token_filters = [POSKeepFilter(['名詞'])]
    #token_filters = [POSStopFilter(['接続詞', '接頭辞', '接尾辞', '記号', '助詞', '助動詞']), TokenCountFilter()]
    a = Analyzer(char_filters, tokenizer, token_filters=token_filters)

    df.loc[df['tags']=="[none]", 'tags'] = \
    df['title'][df['tags']=="[none]"].str.lower().apply(lambda x: "|".join([word.surface for word in a.analyze(x)]))
    
    # ------------------------------------------
    # 比率
    # ------------------------------------------
    df['likes_dislikes_ratio'] = df['likes']/(df['dislikes']+1)
    df['comment_count_likes_ratio'] = df['comment_count']/(df['likes']+1)
    df['comment_count_dislikes_ratio'] = df['comment_count']/(df['dislikes']+1)

    features['num'] += ['likes_dislikes_ratio', 'comment_count_likes_ratio', 'comment_count_dislikes_ratio']

    # ------------------------------------------
    # 積算
    # ------------------------------------------
    df['mul_likes_comments_disabled'] = df['likes'] * df['comments_disabled']
    df['mul_dislikes_comments_disabled'] = df['dislikes'] * df['comments_disabled']
    df['mul_comment_count_ratings_disabled'] = df['comment_count'] * df['ratings_disabled']

    features['num'] += ['mul_likes_comments_disabled', 'mul_dislikes_comments_disabled', 'mul_comment_count_ratings_disabled']
    
    # ------------------------------------------
    # 出現頻度
    # ------------------------------------------
    for col in ['channelId', 'channelTitle', 'categoryId']:
        df['_'.join(list(map(str, ['freq', col])))] = df[col].map(df[col].value_counts())
        
        features['num'] += ['_'.join(list(map(str, ['freq', col])))]

    # ------------------------------------------
    # 'n_tags'数特徴量の生成
    # ------------------------------------------    
    df['n_tags'] = df['tags'].astype(str).apply(lambda x: len(x.split("|")))
    features['num'] += ['n_tags']
    
    # ------------------------------------------
    # 'description'関連の特徴量生成
    # ------------------------------------------    
    df['http_count_in_desc'] = df['description'].apply(lambda x: x.lower().count("http"))
    df['len_description'] = df['description'].apply(lambda x: len(x))
    df['len_title'] = df['title'].apply(lambda x: len(x))

    features['num'] += ['http_count_in_desc', 'len_description', 'len_title']
    
    # ------------------------------------------
    # 'title'/'tag'/'description'内の記述言語関連特徴量の生成
    # ------------------------------------------    
    def checkJapanese(word):
        for ch in word:
            try:
                name = unicodedata.name(ch) 
                if "CJK UNIFIED" in name \
                or "HIRAGANA" in name \
                or "KATAKANA" in name:
                    return True
            except:
              continue
        return False

    def checkAlnum(word):
        alnum = re.compile(r'^[a-zA-Z0-9]+$')
        result = alnum.match(word) is not None
        return result

    # is japanese
    df['isJa_title'] = df['title'].apply(lambda x: checkJapanese(x))
    df['isJa_tags'] = df['tags'].apply(lambda x: checkJapanese(x))
    df['isJa_description'] = df['description'].apply(lambda x: checkJapanese(x))

    features['ohe'] += ['isJa_title', 'isJa_tags', 'isJa_description']
    
    # isalnum
    df['isalnum_title'] = df['title'].apply(lambda x: checkAlnum(x))
    df['isalnum_tags'] = df['tags'].apply(lambda x: checkAlnum(x))
    df['isalnum_description'] = df['description'].apply(lambda x: checkAlnum(x))

    features['ohe'] += ['isalnum_title', 'isalnum_tags', 'isalnum_description']
    
    # cotain english
    df['inclEn_title'] = df['title'].apply(lambda x: len(re.findall(r'[a-zA-Z0-9]', x.lower())))
    df['inclEn_tags'] = df['tags'].apply(lambda x: len(re.findall(r'[a-zA-Z0-9]', x.lower())))
    df['inclEn_description'] = df['description'].apply(lambda x: len(re.findall(r'[a-zA-Z0-9]', x.lower())))
    
    features['num'] += ['inclEn_title', 'inclEn_tags', 'inclEn_description']

    # ------------------------------------------
    # 投稿時期、採取時期からの期間、日時関連特徴量の生成
    # ------------------------------------------    
    # publishedAt
    df['publishedAt'] = pd.to_datetime(df['publishedAt'], utc=True)
    df['publishedAt_year'] = df['publishedAt'].apply(lambda x: x.year)
    df['publishedAt_month'] = df['publishedAt'].apply(lambda x: x.month)
    df['publishedAt_day'] = df['publishedAt'].apply(lambda x: x.day)
    df['publishedAt_hour'] = df['publishedAt'].apply(lambda x: x.hour)
    df['publishedAt_minute'] = df['publishedAt'].apply(lambda x: x.minute)
    df['publishedAt_second'] = df['publishedAt'].apply(lambda x: x.second)
    df['publishedAt_dayofweek'] = df['publishedAt'].apply(lambda x: x.dayofweek)

    df['collection_date'] = \
    df['collection_date'].map(lambda x: x.split('.')).map(lambda x: '20'+x[0]+'-'+x[2]+'-'+x[1]+'T00:00:00.000Z')
        
    # collection_date
    df['collection_date'] = pd.to_datetime(df['collection_date'], utc=True)
    df['collection_date_year'] = df['collection_date'].apply(lambda x: x.year)
    df['collection_date_month'] = df['collection_date'].apply(lambda x: x.month)
    df['collection_date_day'] = df['collection_date'].apply(lambda x: x.day)

    # delta
    df['delta'] = (df['collection_date'] - df['publishedAt']).apply(lambda x: x.days)
    df['log_delta'] = np.log(df['delta'])
    df['sqrt_delta'] = np.sqrt(df['delta'])
    df['pow_delta'] = pow(df['delta'], 2)
    df['log_pow_delta'] = pow(np.log(df['delta']), 2)
    df['publishedAt_delta'] = (df['publishedAt'] - df['publishedAt'].min()).apply(lambda x: x.days)
    df['collection_delta'] = (df['collection_date'] - df['collection_date'].min()).apply(lambda x: x.days)
    
    features['cat'] += ['publishedAt_year', 'publishedAt_month', 'publishedAt_day', \
                        'publishedAt_hour', 'publishedAt_minute', 'publishedAt_second', 'publishedAt_dayofweek', \
                        'collection_date_year', 'collection_date_month', 'collection_date_day']
    
    features['num'] += ['delta', 'log_delta', 'sqrt_delta', 'pow_delta', 'log_pow_delta', \
                        'publishedAt_delta', 'collection_delta']
    
    return df, features
    
def create_features2(df, features, cols_groupby, cols_transform, target_func, option):
    # ------------------------------------------
    # 'cols_groupby'ごとの特徴量生成
    # ------------------------------------------   
    
    cols_to_transform = list(set(cols_transform) - set(cols_groupby))

    #cols_to_transform = [c for c in df.columns if ('likes' in c) | ('dislikes' in c) | ('comment_count' in c)]
    for col_base in cols_groupby:
        for col in cols_to_transform:
            for func in target_func:
                df['_'.join(list(map(str, [col_base, col, func])))] = df.groupby(col_base)[col].transform(func)
                features['num'] += ['_'.join(list(map(str, [col_base, col, func])))]
                
                if option['log']:
                    df['_'.join(list(map(str, [col_base, col, 'log', func])))] = np.log1p(df.groupby(col_base)[col].transform(func))
                    features['num'] += ['_'.join(list(map(str, [col_base, col, 'log', func])))]
                if option['sqrt']:
                    df['_'.join(list(map(str, [col_base, col, 'sqrt', func])))] = np.sqrt(df.groupby(col_base)[col].transform(func))
                    features['num'] += ['_'.join(list(map(str, [col_base, col, 'sqrt', func])))]
                if option['sqrt_log']:
                    df['_'.join(list(map(str, [col_base, col, 'sqrt_log', func])))] = np.log1p(np.sqrt(df.groupby(col_base)[col].transform(func)))
                    features['num'] += ['_'.join(list(map(str, [col_base, col, 'sqrt_log', func])))]
                if option['log_sqrt']:
                    df['_'.join(list(map(str, [col_base, col, 'log_sqrt', func])))] = np.sqrt(np.log1p(df.groupby(col_base)[col].transform(func)))
                    features['num'] += ['_'.join(list(map(str, [col_base, col, 'log_sqrt', func])))]
                if option['pow']:
                    df['_'.join(list(map(str, [col_base, col, 'pow', func])))] = pow(df.groupby(col_base)[col].transform(func), 2)
                    features['num'] += ['_'.join(list(map(str, [col_base, col, 'pow', func])))]
                if option['pow_log']:
                    df['_'.join(list(map(str, [col_base, col, 'pow_log', func])))] = 2*np.log1p(df.groupby(col_base)[col].transform(func))
                    features['num'] += ['_'.join(list(map(str, [col_base, col, 'pow_log', func])))]
                if option['log_pow']:
                    df['_'.join(list(map(str, [col_base, col, 'log_pow', func])))] = pow(np.log1p(df.groupby(col_base)[col].transform(func)), 2)
                    features['num'] += ['_'.join(list(map(str, [col_base, col, 'log_pow', func])))]

    return df, features

def create_features3(df, features, cols_transform, option):
    for col in cols_transform:
        if option['log']:
            df['_'.join(list(map(str, ['log', col])))] = np.log1p(df[col])
            features['num'] += ['_'.join(list(map(str, ['log', col])))]

        if option['sqrt']:
            df['_'.join(list(map(str, ['sqrt', col])))] = np.sqrt(df[col])
            features['num'] += ['_'.join(list(map(str, ['sqrt', col])))]

        if option['sqrt_log']:
            df['_'.join(list(map(str, ['sqrt', 'log', col])))] = np.log1p(np.sqrt(df[col]))
            features['num'] += ['_'.join(list(map(str, ['sqrt', 'log', col])))]
        
        if option['log_sqrt']:
            df['_'.join(list(map(str, ['log', 'sqrt', col])))] = np.sqrt(np.log1p(df[col]))
            features['num'] += ['_'.join(list(map(str, ['log', 'sqrt', col])))]
        
        if option['pow']:
            df['_'.join(list(map(str, ['pow', col])))] = pow(df[col], 2)
            features['num'] += ['_'.join(list(map(str, ['pow', col])))]
        
        if option['pow_log']:
            df['_'.join(list(map(str, ['pow', 'log', col])))] = np.log1p(pow(df[col], 2))
            features['num'] += ['_'.join(list(map(str, ['pow', 'log', col])))]
        
        if option['log_pow']:
            df['_'.join(list(map(str, ['log', 'pow', col])))] = pow(np.log1p(df[col]), 2)
            features['num'] += ['_'.join(list(map(str, ['log', 'pow', col])))]

    return df, features

def create_features4(df, features, option):
    tokenizer = Tokenizer()

    feats_increased = []
    
    title_words = ["video", "official", "music", "公式"]
    tags_words = ["music", "video", "official", "song", "remastered", "vevo", "lyric", "rock", "you", "pop", "live", "queen"]
    desc_words= ["http", "www", "smarturl", "super", "simple", "video", "music", "facebook", "youtube", "twitter", "official", "instagram"]
    
    for j, (title_sentence, tags_sentence, desc_sentence) in enumerate(tqdm(zip(df['title'].str.lower(), df['tags'].str.lower(), df['description'].str.lower()))):
        if option['title']:
            title_text = " ".join(tokenizer.tokenize(title_sentence, wakati=True))
            for word in title_words:
                if (word in title_text) | (f"{word}s" in title_text):
                    df.loc[df.index==j, f'title_{word}'] = 1
                    if not f'title_{word}' in feats_increased:
                        feats_increased += [f'title_{word}']
                
        if option['tags']:
            tags_text = " ".join(tokenizer.tokenize(tags_sentence, wakati=True))
            for word in tags_words:
                if (word in tags_text) | (f"{word}s" in tags_text):
                    df.loc[df.index==j, f'tags_{word}'] = 1
                    if not f'tags_{word}' in feats_increased:
                        feats_increased += [f'tags_{word}']

        if option['description']:
            desc_text = " ".join(tokenizer.tokenize(desc_sentence, wakati=True))
            for word in desc_words:
                if (word in desc_text) | (f"{word}s" in desc_text):
                    df.loc[df.index==j, f'desc_{word}'] = 1
                    if not f'desc_{word}' in feats_increased:
                        feats_increased += [f'desc_{word}']

    features['ohe'] += feats_increased
    feats_increased_dict = {k: 0 for k in feats_increased}
    df.fillna(feats_increased_dict, inplace=True)
    df[feats_increased] = df[feats_increased].astype('int')
        
    return df, features

# ラベルエンコーディング

In [ ]:
def label_encoder(df, cols_to_encode=[]):
    lbl_enc_columns = cols_to_encode #cat_features + date_features + ohe_features

    # Transforming all the labels of all variables
    label_encoders = [LabelEncoder() for _ in range(len(lbl_enc_columns))]

    for col, column in enumerate(lbl_enc_columns):
        unique_values = pd.Series(df[column].unique())
        unique_values = unique_values[unique_values.notnull()]
        label_encoders[col].fit(unique_values)
        df.loc[df[column].notnull(), column] = label_encoders[col].transform(df.loc[df[column].notnull(), column])

    return df

# 標準化

In [ ]:
def standardization(df, fnc_name='Standard', cols_to_std=[]):
    fnc_scaler = {'Standard': StandardScaler(copy=True, with_mean=True, with_std=False),
                  'MinMax': MinMaxScaler(feature_range=(-1, 1), copy=True),
                  'MaxAbs': MaxAbsScaler(copy=True),
                  'Normalize': Normalizer(norm='max'),
                  'Robust': RobustScaler(with_centering=True, with_scaling=True, quantile_range=(25.0, 75.0), copy=True),
                  'Quantile': QuantileTransformer(n_quantiles=1000, output_distribution='normal', ignore_implicit_zeros=False, \
                                                  subsample=100000, random_state=None, copy=True),
                  #'box-cox': PowerTransformer(method='box-cox'),
                  'yeo': PowerTransformer(method='yeo-johnson', standardize=True, copy=True)
                 }
    scaler = fnc_scaler[fnc_name]
    df[cols_to_std] = scaler.fit_transform(df[cols_to_std])

    return df

# featuresの削除

In [ ]:
def drop_features(df, features, features_to_drop):
    cols_to_drop = features_to_drop['num']+features_to_drop['cat']+features_to_drop['date']+features_to_drop['ohe']
    df.drop(cols_to_drop, axis=1, inplace=True)

    for col in features_to_drop['num']:
        features['num'].remove(col)

    for col in features_to_drop['cat']:
        features['cat'].remove(col)

    for col in features_to_drop['date']:
        features['date'].remove(col)

    for col in features_to_drop['ohe']:
        features['ohe'].remove(col)

    return df, features

# 学習、予測の共通処理

In [ ]:
DIFF_THRESHOLD = 5
################################################################################
# METRICS
################################################################################
def rmsle(y, pred_y):
    return mean_squared_error(y, pred_y, squared=False)
    
################################################################################
# CROSS-VALIDATION
################################################################################
def print_cv_scores(label, cv_scores):
    print("*"*40)
    print(f"type(cv_scores): {type(cv_scores)}")
    print(f"{label} cv scores : {cv_scores}")
    print(f"{label} cv mean score : {np.mean(cv_scores)}")
    print(f"{label} cv std score : {np.std(cv_scores)}")
    
def run_cv_model(train, test, target, target_skf, encoding, model_fn, params={}, 
                 eval_fn=None, label='model', cv=5,  repeats=5, seed=43):

    if repeats==1:
        if target_skf is None:
            kf = KFold(n_splits=cv, shuffle=True, random_state=seed)
            target_y = target
        else:
            kf = StratifiedKFold(n_splits=cv, shuffle=True, random_state=seed)
            target_y = target_skf
        divide_counts = cv
    else:
        if target_skf is None:
            kf = RepeatedKFold(n_splits=cv,n_repeats=repeats, random_state=seed)
            target_y = target
        else:
            kf = RepeatedStratifiedKFold(n_splits=cv, n_repeats=repeats, random_state=seed)
            target_y = target_skf
        divide_counts = kf.get_n_splits()
        
    cv_scores = []
    pred_full_test = 0
    pred_train = np.zeros((train.shape[0]))

#    for fold_id, (train_idx, val_idx) in enumerate(kf.split(train, target)):
    for fold_id, (train_idx, val_idx) in enumerate(kf.split(train, target_y)):
        print("*"*40)
        print(f"Started {label} fold:{fold_id+1} / {divide_counts}")
        #tr_X, val_X = train[train_idx], train[val_idx]
        #tr_y, val_y = target[train_idx], target[val_idx]
        tr_X, val_X = train.iloc[train_idx].copy(), train.iloc[val_idx].copy()
        tr_y, val_y = target.iloc[train_idx], target.iloc[val_idx]
        #print(Counter(tr_y), Counter(val_y))
        
        # TARGET ENCODING
        if encoding:
            for c in encoding:
                # 学習データ全体で各カテゴリにおけるtargetの平均を計算
                data_tmp = pd.DataFrame({c: tr_X[c], 'target': tr_y})
                target_mean = data_tmp.groupby(c)['target'].mean()
                # バリデーションデータのカテゴリを置換
                val_X.loc[:, c] = val_X[c].map(target_mean)

                # 学習データの変換後の値を格納する配列を準備
                tmp = np.repeat(np.nan, tr_X.shape[0])
                kf_encoding = KFold(n_splits=4, shuffle=True, random_state=seed)
                for idx_1, idx_2 in kf_encoding.split(tr_X):
                    # out-of-foldで各カテゴリにおける目的変数の平均を計算
                    target_mean = data_tmp.iloc[idx_1].groupby(c)['target'].mean()
                    # 変換後の値を一次配列に格納
                    tmp[idx_2] = tr_X[c].iloc[idx_2].map(target_mean)


                tr_X.loc[:, c] = tmp
        # TARGET ENCODING
        
        params2 = params.copy() 
        model, pred_val_y, pred_test_y = model_fn(
            tr_X, tr_y, val_X, val_y, test, params2)
        
        pred_full_test = pred_full_test + pred_test_y
        pred_train[val_idx] = pred_val_y
        if eval_fn is not None:
            cv_score = eval_fn(val_y, pred_val_y)
            cv_scores.append(cv_score)
            print(f"{label} cv score {fold_id+1}: {cv_score}")
            
    
    print_cv_scores(label, cv_scores)    
    pred_full_test = pred_full_test / divide_counts
    results = {'label': label,
               'train': pred_train, 
               'test': pred_full_test, 
               'cv': cv_scores}
    
    RMSLE = mean_squared_error(target.values, results['train'], squared=False)
    print(f"Overall RMSLE={RMSLE}")
    
    return results

################################################################################
# RESULTS
################################################################################
def submit_results(target, results, test_id, MODEL):
    RMSLE = mean_squared_error(target.values, results['train'], squared=False)
    #print(f"Overall RMSLE={RMSLE}")

    # Make submission
    print("Saving submission file")
    submission = pd.DataFrame({'id': test_id, 'y': np.expm1(results['test'])})
    submission.to_csv(f"./{out_dir}/submission_{MODEL}_CV{RMSLE:.6f}.csv", index=False)
        
    return submission

def check_results(y, results):
    y_diff = np.abs(np.expm1(y) - np.expm1(results['train']))
    y_log1p_diff = np.abs(y - results['train'])
    display(y_diff[y_log1p_diff>DIFF_THRESHOLD].index.values)
    display(train_data[y_log1p_diff>DIFF_THRESHOLD])
    display(pd.concat([pd.DataFrame(y[y_log1p_diff>DIFF_THRESHOLD], columns=['y']), \
                       pd.DataFrame(results['train'][y_log1p_diff>DIFF_THRESHOLD], \
                                    index=y_diff[y_log1p_diff>DIFF_THRESHOLD].index.values, columns=['pred_train'])], axis=1))    

    get_hist4(results['train'], "pred_train", \
              y, "y", \
              results['test'], "pred_test", \
              y_log1p_diff, "diff")
    
    display(pd.concat([pd.DataFrame(results['train'], columns=['pred_train']), \
                       pd.DataFrame(y, columns=['y']), \
                       y_log1p_diff.rename('y_log1p_diff')], \
                       axis=1).describe())
    
    display(pd.DataFrame(results['test'], columns=['pred_test']).describe())
    
    RMSLE = mean_squared_error(y, results['train'], squared=False)
    display(f"Overall RMSLE={RMSLE:.6f}")
    
################################################################################
# MODEL
################################################################################
def runLGB(train_X, train_y, val_X, val_y, test_X, params):
    model = lgb.LGBMRegressor(**params)
    model.fit(train_X, train_y, eval_set=(val_X, val_y), early_stopping_rounds=500, eval_metric='rmse', verbose=500)
    pred_val_y = model.predict(val_X)
    pred_test_y = model.predict(test_X)
    return model, pred_val_y, pred_test_y

def runXGB(train_X, train_y, val_X, val_y, test_X, params):
    model = xgb.XGBRegressor(**params)
    model.fit(train_X, train_y, eval_set=[[val_X, val_y]], early_stopping_rounds=500, eval_metric='rmse', verbose=500)
    pred_val_y = model.predict(val_X)
    pred_test_y = model.predict(test_X)
    return model, pred_val_y, pred_test_y

def runCAT(train_X, train_y, val_X, val_y, test_X, params):
    model = CatBoostRegressor(**params)
    model.fit(train_X, train_y, eval_set=(val_X, val_y), early_stopping_rounds=500, use_best_model=True, verbose=500)
    pred_val_y = model.predict(val_X)
    pred_test_y = model.predict(test_X)
    return model, pred_val_y, pred_test_y

################################################################################
# MODEL PARAMETERS
################################################################################
lgb_params = {'boosting_type': 'gbdt', 'tree_learner': 'feature', #''serial' or feature' or 'data' or 'voting'
              'num_leaves': 31, 'max_depth': -1,
              'learning_rate': 5e-2, 'n_estimators': 20000, 'importance_type': 'gain',
              'subsample_for_bin': 200000, 'objective': 'rmse', 'min_split_gain': 0.0, 'min_child_weight': 1e-3, 'min_child_samples': 20, 
              'bagging_freq': 0, 'bagging_fraction': 1.0, 'feature_fraction': 1.0,
              'reg_alpha': 0.2, 'reg_lambda': 0.2,
              'random_state': 43, 'data_random_seed': 1,
              'n_jobs': -1, 'silent': False}

xgb_params = {'base_score': 0.5, 'booster': 'gbtree', 'colsample_bylevel': 1, 'colsample_bynode': 1, 'colsample_bytree': 1, 'gamma': 0,
              'learning_rate': 5e-2, 'n_estimators': 20000, 'importance_type': 'gain',
              'max_delta_step': 0, 'max_depth': 6, 'min_child_weight': 0,
              'objective': 'reg:squarederror', 'reg_alpha': 0.2, 'reg_lambda': 0.2, 'scale_pos_weight': 1,
              'subsample': 0.9,
              'silent': None, 'verbosity': 0,
              'random_state': 43, 'seed': 43,
              'tree_method': 'gpu_hist', 'gpu_id': 0}

cat_params = {'iterations':20000, 'depth': 6, 'boosting_type': 'Ordered', #'Ordered', #'Plain',
              'loss_function': 'RMSE', 'eval_metric': 'RMSE',
              'learning_rate': 5e-2, 'leaf_estimation_method': 'Gradient', #'Newton', 'Exact'
              'l2_leaf_reg': 1.0, 'random_strength': 1.0, 'bagging_temperature': 1.0, 'has_time': False,
              'grow_policy': 'SymmetricTree', #'Depthwise', 'Lossguide'
              'min_data_in_leaf': 1, 'max_leaves': 31,
              'random_seed': 43,
#              'one_hot_max_size': len(cat_features),
              'task_type': 'GPU'}



# 学習/予測用データの準備 (#1)

In [ ]:
df = pd.concat([train, test]).reset_index(drop=True)

df.loc[df['categoryId']==43, 'categoryId'] = 30

In [ ]:
def check_features(df, features):
    print("-"*40)
    print(f"実特徴量数: {len(df.columns)} / 計算上の特徴量数: {len(features['num'])+len(features['cat'])+len(features['date'])+len(features['ohe'])}")
    print("-"*40)
    print(f"(内訳) num_features: {len(features['num'])}, cat_features: {len(features['cat'])}, date_features: {len(features['date'])}, ohe_features: {len(features['ohe'])}")
    print("-"*40)

    return

In [ ]:
%%time

seed_everything(seed=47)

df = pd.concat([train, test]).reset_index(drop=True)

df.loc[df['categoryId']==43, 'categoryId'] = 30

# 特徴量の分類
features = {'cat': ['video_id', 'title', 'channelId', 'channelTitle', 'categoryId', 'tags', 'thumbnail_link', 'description', 'comments_disabled', 'ratings_disabled'],
            'date': ['publishedAt', 'collection_date'],
            'num': ['likes', 'dislikes', 'comment_count'],
            'ohe': []
           }

# ラベルエンコーディング
print("ラベルエンコーディング")
df = label_encoder(df, cols_to_encode=['categoryId'])

# 特徴量生成
print("特徴量生成")
df, features = create_features(df, features)

# ラベルエンコーディング
print("ラベルエンコーディング")
df = label_encoder(df, cols_to_encode=features['cat']+features['date'])

print("特徴量生成2 categoryId")
df, features = create_features2(df, features,
                                cols_groupby=['categoryId'],
                                cols_transform=['likes', 'dislikes', 'comment_count', 'channelId'],
                                target_func=['max', 'min', 'mean'],
                                option={"log": False,\
                                        "sqrt": False, "sqrt_log": False, "log_sqrt": False,\
                                        "pow": False, "pow_log": False, "log_pow": False})
print("特徴量生成2 comments_disabled")
df, features = create_features2(df, features,
                                cols_groupby=['comments_disabled'],
                                cols_transform=['likes'],
                                target_func=['max', 'min', 'mean'],
                                option={"log": False,\
                                        "sqrt": False, "sqrt_log": False, "log_sqrt": False,\
                                        "pow": False, "pow_log": False, "log_pow": False})
print("特徴量生成2 channelId")
df, features = create_features2(df, features,
                                cols_groupby=['channelId'],
                                cols_transform=list(set(features['num'] + features['cat'] + features['ohe'])),
                                target_func=['max', 'min', 'mean'],
                                option={"log": False, \
                                        "sqrt": False, "sqrt_log": False, "log_sqrt": False,\
                                        "pow": False, "pow_log": False, "log_pow": False})
print("特徴量生成3")
df, features = create_features3(df, features,
                                cols_transform=[c for c in df.columns if (('likes' in c)|('dislikes' in  c)|('comment_count' in c))&
                                               (('diff_likes' not in c)&('diff_dislikes' not in c)&('diff_comments' not in c))],
                                option={"log": False,\
                                        "sqrt": True, "sqrt_log": False, "log_sqrt": False,\
                                        "pow": True, "pow_log": False, "log_pow": True})

# 特徴量数のチェック
print("特徴量のチェック")
check_features(df, features)

# df/featuresの退避
df_save = df.copy()
features_save = {"cat":  features["cat"].copy(),
                 "date": features["date"].copy(),
                 "num":  features["num"].copy(),
                 "ohe":  features["ohe"].copy()}

# 削除する特徴量のリスト
feats_to_drop_list = []
if DATASET >= 1:
    feats_to_drop_list += [['len_description', 'len_title',
                           'channelId_channelTitle_max',  'channelId_channelTitle_min', 'channelId_channelTitle_mean',
                           'channelId_len_description_max',  'channelId_len_description_min', 'channelId_len_description_mean',
                           'channelId_len_title_max', 'channelId_len_title_min', 'channelId_len_title_mean',
                           'channelId_tags_max', 'channelId_tags_min', 'channelId_tags_mean',
                           'inclEn_title', 'inclEn_description']]
if DATASET >= 2:
    feats_to_drop_list += [['len_description', 'len_title',
                           #'channelId_channelTitle_max',  'channelId_channelTitle_min', 'channelId_channelTitle_mean',
                           'channelId_len_description_max',  'channelId_len_description_min', 'channelId_len_description_mean',
                           'channelId_len_title_max', 'channelId_len_title_min', 'channelId_len_title_mean',
                           'channelId_tags_max', 'channelId_tags_min', 'channelId_tags_mean',
                           'inclEn_title', 'inclEn_description']]
if DATASET >= 3:
    feats_to_drop_list += [['len_description', 'len_title',
                           'channelId_channelTitle_max',  'channelId_channelTitle_min', 'channelId_channelTitle_mean',
                           #'channelId_len_description_max',  'channelId_len_description_min', 'channelId_len_description_mean',
                           'channelId_len_title_max', 'channelId_len_title_min', 'channelId_len_title_mean',
                           'channelId_tags_max', 'channelId_tags_min', 'channelId_tags_mean',
                           'inclEn_title', 'inclEn_description']]
if DATASET >=4:
    feats_to_drop_list += [['len_description', 'len_title',
                           'channelId_channelTitle_max',  'channelId_channelTitle_min', 'channelId_channelTitle_mean',
                           'channelId_len_description_max',  'channelId_len_description_min', 'channelId_len_description_mean',
                           #'channelId_len_title_max', 'channelId_len_title_min', 'channelId_len_title_mean',
                           'channelId_tags_max', 'channelId_tags_min', 'channelId_tags_mean',
                           'inclEn_title', 'inclEn_description']]
if DATASET >=5:
    feats_to_drop_list += [['len_description', 'len_title',
                           'channelId_channelTitle_max',  'channelId_channelTitle_min', 'channelId_channelTitle_mean',
                           'channelId_len_description_max',  'channelId_len_description_min', 'channelId_len_description_mean',
                           'channelId_len_title_max', 'channelId_len_title_min', 'channelId_len_title_mean',
                           #'channelId_tags_max', 'channelId_tags_min', 'channelId_tags_mean',
                           'inclEn_title', 'inclEn_description']]

# 特徴量バリエーションを含むtrain/testデータのリスト
X_train_list = []
X_test_list  = []

for feats_to_drop_num in tqdm(feats_to_drop_list):
    # df/featuresの復帰
    df = df_save.copy()
    features["cat"]  = features_save["cat"].copy()
    features["date"] = features_save["date"].copy()
    features["ohe"]  = features_save["ohe"].copy()
    features["num"]  = features_save["num"].copy()

    # 特徴量削除
    print("特徴量削除")
    feats_to_drop = {"cat": ['video_id', 'channelId', 'title', 'channelTitle', 'tags', 'thumbnail_link', 'description'],
                     "date": ['publishedAt', 'collection_date'],
                     "ohe": [],
                     "num": []}

    #-----------------------------------------------------------------------------------------------------------
    # CV: 0.725691, LB: 0.723
    #-----------------------------------------------------------------------------------------------------------
    feats_to_drop["cat"] += ['publishedAt_second', 'publishedAt_minute', 'publishedAt_hour', 'publishedAt_day', 'publishedAt_dayofweek']
    feats_to_drop["num"] += feats_to_drop_num

    #-----------------------------------------------------------------------------------------------------------

    df, features = drop_features(df, features, feats_to_drop)

    # nunique()==1の特徴量を削除
    for col in df.loc[:, df.nunique()==1].columns:
        features["num"].remove(col)
    df.drop(df.loc[:, df.nunique()==1].columns, axis=1, inplace=True)

    # データ型変換
    print("データ型変換")
    #df[features["num"]] = df[features["num"]].astype('float32')
    df = df.astype('float')
    df[features["cat"]] = df[features["cat"]].astype('int')
    df[features["ohe"]] = df[features["ohe"]].astype('int')

    # 標準化
    print("標準化")
    #df = standardization(df, fnc_name="Standard", cols_to_std=features["num"])
    df = standardization(df, fnc_name="MinMax", cols_to_std=features["num"])

    # 特徴量数のチェック
    #print("特徴量のチェック")
    check_features(df, features)

    # 学習、予測データ分割
    X_train = df.iloc[:y.shape[0], :].reset_index(drop=True).copy()
    X_test  = df.iloc[y.shape[0]:, :].reset_index(drop=True).copy()

    # 欠損値、無限大/無限小値有無のチェック
    print(f"学習データ中の欠損値数: {X_train.isnull().sum().sum()}")
    print(f"学習データ中の無限値数: {np.count_nonzero(np.isinf(X_train))}")
    print(f"予測データ中の欠損値数: {X_test.isnull().sum().sum()}")
    print(f"予測データ中の無現値数: {np.count_nonzero(np.isinf(X_test))}")

    X_train_list.append(X_train)
    X_test_list.append(X_test)

In [ ]:
%%time

for X_train, X_test in tqdm(zip(X_train_list, X_test_list)):
    display(description(X_train))
    display(description(X_test))

# 3x Single Model
### LGBMRegressor, XGBRegressor, CatBoostRegressor
---

In [ ]:
%%time

def output_results(results, test_id, seed, N_FOLDS, dataid, out_dir):
    submission = submit_results(y, results, test_id, f"{results['label']}_SEED{seed}_FOLDS{N_FOLDS}_0624")
    with open(f"./{out_dir}/{results['label']}_train_SEED{seed}_FOLDS{N_FOLDS}_0624.pickle", 'wb') as f:
        pickle.dump(results['train'], f)
    with open(f"./{out_dir}/{results['label']}_test_SEED{seed}_FOLDS{N_FOLDS}_0624.pickle", 'wb') as f:
        pickle.dump(results['test'], f)
    return submission

if LGB:
    lgb_results_l = []
if XGB:
    xgb_results_l = []
if CAT:
    cat_results_l = []
    
for dataid, (X_train, X_test) in enumerate(tqdm(zip(X_train_list, X_test_list))):
    X_train_cmntf = X_train[(X_train['comments_disabled']==False)]
    y_cmntf       = y[(X_train['comments_disabled']==False)]
    y_bin_cmntf   = y_bin[(X_train['comments_disabled']==False)]
    X_test_cmntf  = X_test[(X_test['comments_disabled']==False)]

    X_train_rest = X_train[(X_train['comments_disabled']==True)]
    X_test_rest  = X_test[(X_test['comments_disabled']==True)]
    y_rest       = y[(X_train['comments_disabled']==True)]

    seed_l = [47]
    N_FOLDS, encoding = 8, []
    
    for k, seed in enumerate(tqdm(seed_l)):
        print("*"*80)
        print(f"seed = {seed}")
        print("*"*80)

        if LGB:
            #-----------------------------------------------
            # LGB MODEL 1
            #-----------------------------------------------
            # df[df['comments_disabled']==False]の学習、予測
            lgb_params = {'boosting_type': 'gbdt',
                          'objective' : 'regression',
                          'metric' : 'rmse',  
                          'tree_learner': 'feature', #''serial' or feature' or 'data' or 'voting'
                          'max_depth': 16,
                          'min_child_samples': 10, 
                          'min_split_gain': 0.01,
                          'min_child_weight': 1e-2,
                          'reg_alpha': 0.1,
                          'reg_lambda': 1,
                          'num_leaves': 35,
                          'max_bin': 300,
                          'learning_rate': 1e-2,
                          'bagging_fraction': 0.9,
                          'bagging_freq': 1,
                          'bagging_seed': 4590,
                          'feature_fraction': 0.85,
                          'n_estimators': 50000,
                          'importance_type': 'gain',
                          'subsample_for_bin': 200000,
                          'random_state': seed,
                          'data_random_seed': seed,
                          'n_jobs': -1,
                          'silent': False}
            if SKF:
                lgb_results_cmntf = run_cv_model(train=X_train_cmntf, test=X_test_cmntf, target=y_cmntf, target_skf=y_bin_cmntf, encoding=[],
                                                 model_fn=runLGB, params=lgb_params, eval_fn=rmsle, label='LGBMRegressor',
                                                 cv=N_FOLDS, repeats=1, seed=seed)
                # df全体の学習、予測
                lgb_results = run_cv_model(train=X_train, test=X_test, target=y, target_skf=y_bin, encoding=[],
                                           model_fn=runLGB, params=lgb_params, eval_fn=rmsle, label='LGBMRegressor',
                                           cv=N_FOLDS, repeats=1, seed=seed)
            else:
                lgb_results_cmntf = run_cv_model(train=X_train_cmntf, test=X_test_cmntf, target=y_cmntf, target_skf=None, encoding=[],
                                                 model_fn=runLGB, params=lgb_params, eval_fn=rmsle, label='LGBMRegressor',
                                                 cv=N_FOLDS, repeats=1, seed=seed)
                # df全体の学習、予測
                lgb_results = run_cv_model(train=X_train, test=X_test, target=y, target_skf=None, encoding=[],
                                           model_fn=runLGB, params=lgb_params, eval_fn=rmsle, label='LGBMRegressor',
                                           cv=N_FOLDS, repeats=1, seed=seed)
                
            lgb_results_merged = {
                "label": "LGBMRegressor",
                "train": pd.concat([pd.Series(lgb_results_cmntf['train'], index=X_train_cmntf.index),
                                    pd.Series(lgb_results['train'][X_train_rest.index], index=X_train_rest.index)]).sort_index(),
                "test":  pd.concat([pd.Series(lgb_results_cmntf['test'], index=X_test_cmntf.index),
                                    pd.Series(lgb_results['test'][X_test_rest.index], index=X_test_rest.index)]).sort_index()}

            print("-"*80)
            RMSLE_base    = mean_squared_error(y, lgb_results['train'], squared=False)
            print(f"Base    RMSLE={RMSLE_base}")
            RMSLE_base_false = mean_squared_error(y_cmntf, lgb_results['train'][X_train_cmntf.index], squared=False)
            print(f"f in B  RMSLE={RMSLE_base_false}")
            RMSLE_false   = mean_squared_error(y_cmntf, lgb_results_cmntf['train'], squared=False)
            print(f"False   RMSLE={RMSLE_false}")
            RMSLE_merged  = mean_squared_error(y, lgb_results_merged['train'], squared=False)
            print(f"merged  RMSLE={RMSLE_merged}")
            print("-"*80)

            if RMSLE_base_false > RMSLE_false:
                results = lgb_results_merged
            else:
                results = lgb_results

            y_pred = pd.Series(results['train']).copy()
            y_pred.where((y_pred<14)&(X_train.comments_disabled==True), y_pred*0.99965, inplace=True)
            y_pred.where((y_pred>14)&(X_train.comments_disabled==True), y_pred*1.0001, inplace=True)
            y_pred.where((y_pred<14)&(X_train.ratings_disabled==True), y_pred*0.99965, inplace=True)
            y_pred.where((y_pred>14)&(X_train.ratings_disabled==True), y_pred*1.0001, inplace=True)

            RMSLE_merged  = mean_squared_error(y, y_pred, squared=False)
            print(f"merged2 RMSLE={RMSLE_merged}")
            print("-"*80)

            sunmission = output_results(results, test_id, seed, N_FOLDS, dataid, out_dir)

            #-----------------------------------------------
            # LGB MODEL 2
            #-----------------------------------------------
            # df[df['comments_disabled']==False]の学習、予測
            lgb_params = {'boosting_type': 'gbdt',
                          'objective' : 'regression',
                          'metric' : 'rmse',  
                          'tree_learner': 'feature', #''serial' or feature' or 'data' or 'voting'
                          'max_depth': -1,
                          'min_child_samples': 10, 
                          'min_split_gain': 0.01,
                          'min_child_weight': 1e-2,
                          'reg_alpha': 0.1,
                          'reg_lambda': 1,
                          'num_leaves': 35,
                          'max_bin': 300,
                          'learning_rate': 2e-2,
                          'bagging_fraction': 0.9,
                          'bagging_freq': 1,
                          'bagging_seed': 4590,
                          'feature_fraction': 0.85,
                          'n_estimators': 50000,
                          'importance_type': 'gain',
                          'subsample_for_bin': 200000,
                          'random_state': seed,
                          'data_random_seed': seed,
                          'n_jobs': -1,
                          'silent': False}
            if SKF:
                lgb_results_cmntf = run_cv_model(train=X_train_cmntf, test=X_test_cmntf, target=y_cmntf, target_skf=y_bin_cmntf, encoding=[],
                                                 model_fn=runLGB, params=lgb_params, eval_fn=rmsle, label='LGBMRegressor2',
                                                 cv=N_FOLDS, repeats=1, seed=seed)
                # df全体の学習、予測
                lgb_results = run_cv_model(train=X_train, test=X_test, target=y, target_skf=y_bin, encoding=[],
                                           model_fn=runLGB, params=lgb_params, eval_fn=rmsle, label='LGBMRegressor2',
                                           cv=N_FOLDS, repeats=1, seed=seed)
            else:
                lgb_results_cmntf = run_cv_model(train=X_train_cmntf, test=X_test_cmntf, target=y_cmntf, target_skf=None, encoding=[],
                                                 model_fn=runLGB, params=lgb_params, eval_fn=rmsle, label='LGBMRegressor2',
                                                 cv=N_FOLDS, repeats=1, seed=seed)
                # df全体の学習、予測
                lgb_results = run_cv_model(train=X_train, test=X_test, target=y, target_skf=None, encoding=[],
                                           model_fn=runLGB, params=lgb_params, eval_fn=rmsle, label='LGBMRegressor2',
                                           cv=N_FOLDS, repeats=1, seed=seed)
                
            lgb_results_merged = {
                "label": "LGBMRegressor2",
                "train": pd.concat([pd.Series(lgb_results_cmntf['train'], index=X_train_cmntf.index),
                                    pd.Series(lgb_results['train'][X_train_rest.index], index=X_train_rest.index)]).sort_index(),
                "test":  pd.concat([pd.Series(lgb_results_cmntf['test'], index=X_test_cmntf.index),
                                    pd.Series(lgb_results['test'][X_test_rest.index], index=X_test_rest.index)]).sort_index()}

            print("-"*80)
            RMSLE_base    = mean_squared_error(y, lgb_results['train'], squared=False)
            print(f"Base    RMSLE={RMSLE_base}")
            RMSLE_base_false = mean_squared_error(y_cmntf, lgb_results['train'][X_train_cmntf.index], squared=False)
            print(f"f in B  RMSLE={RMSLE_base_false}")
            RMSLE_false   = mean_squared_error(y_cmntf, lgb_results_cmntf['train'], squared=False)
            print(f"False   RMSLE={RMSLE_false}")
            RMSLE_merged  = mean_squared_error(y, lgb_results_merged['train'], squared=False)
            print(f"merged  RMSLE={RMSLE_merged}")
            print("-"*80)

            if RMSLE_base_false > RMSLE_false:
                results = lgb_results_merged
            else:
                results = lgb_results
            
            y_pred = pd.Series(results['train']).copy()
            y_pred.where((y_pred<14)&(X_train.comments_disabled==True), y_pred*0.99965, inplace=True)
            y_pred.where((y_pred>14)&(X_train.comments_disabled==True), y_pred*1.0001, inplace=True)
            y_pred.where((y_pred<14)&(X_train.ratings_disabled==True), y_pred*0.99965, inplace=True)
            y_pred.where((y_pred>14)&(X_train.ratings_disabled==True), y_pred*1.0001, inplace=True)

            RMSLE_merged  = mean_squared_error(y, y_pred, squared=False)
            print(f"merged2 RMSLE={RMSLE_merged}")
            print("-"*80)

            sunmission = output_results(results, test_id, seed, N_FOLDS, dataid, out_dir)
            
        if XGB:
            #-----------------------------------------------
            # XGB MODEL 1
            #-----------------------------------------------
            # df[df['comments_disabled']==False]の学習、予測
            xgb_params = {'base_score': 0.5,
                          'booster': 'gbtree',
                          'objective': 'reg:squarederror',
                          'colsample_bylevel': 0.6,
                          'colsample_bynode': 0.6,
                          'colsample_bytree': 0.6,
                          'gamma': 0,
                          'learning_rate': 1e-2,
                          'n_estimators': 50000,
                          'importance_type': 'gain',
                          'max_delta_step': 0,
                          'max_depth': 6,
                          'min_child_weight': 0,
                          'reg_alpha': 0.1,
                          'reg_lambda': 1,
                          'scale_pos_weight': 1,
                          'subsample': 0.9,
                          'silent': None,
                          'verbosity': 0,
                          'random_state': seed,
                          'seed': seed,
                          'tree_method': 'gpu_hist',
                          'gpu_id': 0}
            if SKF:
                xgb_results_cmntf = run_cv_model(train=X_train_cmntf, test=X_test_cmntf, target=y_cmntf, target_skf=y_bin_cmntf, encoding=[],
                                                 model_fn=runXGB, params=xgb_params, eval_fn=rmsle, label='XGBRegressor',
                                                 cv=N_FOLDS, repeats=1, seed=seed)
                # df全体の学習、予測
                xgb_results = run_cv_model(train=X_train, test=X_test, target=y, target_skf=y_bin, encoding=[],
                                           model_fn=runXGB, params=xgb_params, eval_fn=rmsle, label='XGBRegressor',
                                           cv=N_FOLDS, repeats=1, seed=seed)
            else:
                xgb_results_cmntf = run_cv_model(train=X_train_cmntf, test=X_test_cmntf, target=y_cmntf, target_skf=None, encoding=[],
                                                 model_fn=runXGB, params=xgb_params, eval_fn=rmsle, label='XGBRegressor',
                                                 cv=N_FOLDS, repeats=1, seed=seed)
                # df全体の学習、予測
                xgb_results = run_cv_model(train=X_train, test=X_test, target=y, target_skf=None, encoding=[],
                                           model_fn=runXGB, params=xgb_params, eval_fn=rmsle, label='XGBRegressor',
                                           cv=N_FOLDS, repeats=1, seed=seed)
                
            xgb_results_merged = {
                "label": "XGBRegressor",
                "train": pd.concat([pd.Series(xgb_results_cmntf['train'], index=X_train_cmntf.index),
                                    pd.Series(xgb_results['train'][X_train_rest.index], index=X_train_rest.index)]).sort_index(),
                "test":  pd.concat([pd.Series(xgb_results_cmntf['test'], index=X_test_cmntf.index),
                                    pd.Series(xgb_results['test'][X_test_rest.index], index=X_test_rest.index)]).sort_index()}
            
            print("-"*80)
            RMSLE_base    = mean_squared_error(y, xgb_results['train'], squared=False)
            print(f"Base    RMSLE={RMSLE_base}")
            RMSLE_base_false = mean_squared_error(y_cmntf, xgb_results['train'][X_train_cmntf.index], squared=False)
            print(f"f in B  RMSLE={RMSLE_base_false}")
            RMSLE_false   = mean_squared_error(y_cmntf, xgb_results_cmntf['train'], squared=False)
            print(f"False   RMSLE={RMSLE_false}")
            RMSLE_merged  = mean_squared_error(y, xgb_results_merged['train'], squared=False)
            print(f"merged  RMSLE={RMSLE_merged}")
            print("-"*80)

            if RMSLE_base_false > RMSLE_false:
                results = xgb_results_merged
            else:
                results = xgb_results
            
            y_pred = pd.Series(results['train']).copy()
            y_pred.where((y_pred<14)&(X_train.comments_disabled==True), y_pred*0.99965, inplace=True)
            y_pred.where((y_pred>14)&(X_train.comments_disabled==True), y_pred*1.0001, inplace=True)
            y_pred.where((y_pred<14)&(X_train.ratings_disabled==True), y_pred*0.99965, inplace=True)
            y_pred.where((y_pred>14)&(X_train.ratings_disabled==True), y_pred*1.0001, inplace=True)

            RMSLE_merged  = mean_squared_error(y, y_pred, squared=False)
            print(f"merged2 RMSLE={RMSLE_merged}")
            print("-"*80)

            sunmission = output_results(results, test_id, seed, N_FOLDS, dataid, out_dir)

            #-----------------------------------------------
            # XGB MODEL 2
            #-----------------------------------------------
            # df[df['comments_disabled']==False]の学習、予測
            xgb_params = {'base_score': 0.5,
                          'booster': 'gbtree',
                          'objective': 'reg:squarederror',
                          'colsample_bylevel': 0.6,
                          'colsample_bynode': 0.6,
                          'colsample_bytree': 0.6,
                          'gamma': 0,
                          'learning_rate': 1e-2,
                          'n_estimators': 50000,
                          'importance_type': 'gain',
                          'max_delta_step': 0,
                          'max_depth': 8,
                          'min_child_weight': 0,
                          'reg_alpha': 0.1,
                          'reg_lambda': 1,
                          'scale_pos_weight': 1,
                          'subsample': 0.8,
                          'silent': None,
                          'verbosity': 0,
                          'random_state': seed,
                          'seed': seed,
                          'tree_method': 'gpu_hist',
                          'gpu_id': 0}
            if SKF:
                xgb_results_cmntf = run_cv_model(train=X_train_cmntf, test=X_test_cmntf, target=y_cmntf, target_skf=y_bin_cmntf, encoding=[],
                                                 model_fn=runXGB, params=xgb_params, eval_fn=rmsle, label='XGBRegressor2',
                                                 cv=N_FOLDS, repeats=1, seed=seed)
                # df全体の学習、予測
                xgb_results = run_cv_model(train=X_train, test=X_test, target=y, target_skf=y_bin, encoding=[],
                                           model_fn=runXGB, params=xgb_params, eval_fn=rmsle, label='XGBRegressor2',
                                           cv=N_FOLDS, repeats=1, seed=seed)
            else:
                xgb_results_cmntf = run_cv_model(train=X_train_cmntf, test=X_test_cmntf, target=y_cmntf, target_skf=None, encoding=[],
                                                 model_fn=runXGB, params=xgb_params, eval_fn=rmsle, label='XGBRegressor2',
                                                 cv=N_FOLDS, repeats=1, seed=seed)
                # df全体の学習、予測
                xgb_results = run_cv_model(train=X_train, test=X_test, target=y, target_skf=None, encoding=[],
                                           model_fn=runXGB, params=xgb_params, eval_fn=rmsle, label='XGBRegressor2',
                                           cv=N_FOLDS, repeats=1, seed=seed)
                
            xgb_results_merged = {
                "label": "XGBRegressor2",
                "train": pd.concat([pd.Series(xgb_results_cmntf['train'], index=X_train_cmntf.index),
                                    pd.Series(xgb_results['train'][X_train_rest.index], index=X_train_rest.index)]).sort_index(),
                "test":  pd.concat([pd.Series(xgb_results_cmntf['test'], index=X_test_cmntf.index),
                                    pd.Series(xgb_results['test'][X_test_rest.index], index=X_test_rest.index)]).sort_index()}
            
            print("-"*80)
            RMSLE_base    = mean_squared_error(y, xgb_results['train'], squared=False)
            print(f"Base    RMSLE={RMSLE_base}")
            RMSLE_base_false = mean_squared_error(y_cmntf, xgb_results['train'][X_train_cmntf.index], squared=False)
            print(f"f in B  RMSLE={RMSLE_base_false}")
            RMSLE_false   = mean_squared_error(y_cmntf, xgb_results_cmntf['train'], squared=False)
            print(f"False   RMSLE={RMSLE_false}")
            RMSLE_merged  = mean_squared_error(y, xgb_results_merged['train'], squared=False)
            print(f"merged  RMSLE={RMSLE_merged}")
            print("-"*80)

            if RMSLE_base_false > RMSLE_false:
                results = xgb_results_merged
            else:
                results = xgb_results
            
            y_pred = pd.Series(results['train']).copy()
            y_pred.where((y_pred<14)&(X_train.comments_disabled==True), y_pred*0.99965, inplace=True)
            y_pred.where((y_pred>14)&(X_train.comments_disabled==True), y_pred*1.0001, inplace=True)
            y_pred.where((y_pred<14)&(X_train.ratings_disabled==True), y_pred*0.99965, inplace=True)
            y_pred.where((y_pred>14)&(X_train.ratings_disabled==True), y_pred*1.0001, inplace=True)

            RMSLE_merged  = mean_squared_error(y, y_pred, squared=False)
            print(f"merged2 RMSLE={RMSLE_merged}")
            print("-"*80)

            sunmission = output_results(results, test_id, seed, N_FOLDS, dataid, out_dir)

        if CAT:
            #-----------------------------------------------
            # CATBOOST MODEL 1
            #-----------------------------------------------
            # df[df['comments_disabled']==False]の学習、予測
            cat_params = {'bootstrap_type': 'Bayesian', 
                          'boosting_type': 'Plain', #'Ordered', #'Plain',
                          'iterations':50000,
                          'depth': 8,
                          'loss_function': 'RMSE',
                          'eval_metric': 'RMSE',
                          'learning_rate': 1e-2,
                          'leaf_estimation_method': 'Gradient', #'Newton', 'Exact'
                          'l2_leaf_reg': 1.0,
                          'random_strength': 0.8,
                          'bagging_temperature': 0.9,
                          'has_time': False,
                          'grow_policy': 'SymmetricTree', #'Depthwise', 'Lossguide'
                          'min_data_in_leaf': 1,
                          'max_leaves': 31,
                          'random_seed': seed,
                          #'one_hot_max_size': len(cat_features),
                          'task_type': 'GPU'}
            if SKF:
                cat_results_cmntf = run_cv_model(train=X_train_cmntf, test=X_test_cmntf, target=y_cmntf, target_skf=y_bin_cmntf, encoding=[],
                                                 model_fn=runCAT, params=cat_params, eval_fn=rmsle, label='CatBoostRegressor',
                                                 cv=N_FOLDS, repeats=1, seed=seed)
                # df全体の学習、予測
                cat_results = run_cv_model(train=X_train, test=X_test, target=y, target_skf=y_bin, encoding=[],
                                           model_fn=runCAT, params=cat_params, eval_fn=rmsle, label='CatBoostRegressor',
                                           cv=N_FOLDS, repeats=1, seed=seed)
            else:
                cat_results_cmntf = run_cv_model(train=X_train_cmntf, test=X_test_cmntf, target=y_cmntf, target_skf=None, encoding=[],
                                                 model_fn=runCAT, params=cat_params, eval_fn=rmsle, label='CatBoostRegressor',
                                                 cv=N_FOLDS, repeats=1, seed=seed)
                # df全体の学習、予測
                cat_results = run_cv_model(train=X_train, test=X_test, target=y, target_skf=None, encoding=[],
                                           model_fn=runCAT, params=cat_params, eval_fn=rmsle, label='CatBoostRegressor',
                                           cv=N_FOLDS, repeats=1, seed=seed)

            cat_results_merged = {
                "label": "CatBoostRegressor",
                "train": pd.concat([pd.Series(cat_results_cmntf['train'], index=X_train_cmntf.index),
                                    pd.Series(cat_results['train'][X_train_rest.index], index=X_train_rest.index)]).sort_index(),
                "test":  pd.concat([pd.Series(cat_results_cmntf['test'], index=X_test_cmntf.index),
                                    pd.Series(cat_results['test'][X_test_rest.index], index=X_test_rest.index)]).sort_index()}

            print("-"*80)
            RMSLE_base    = mean_squared_error(y, cat_results['train'], squared=False)
            print(f"Base    RMSLE={RMSLE_base}")
            RMSLE_base_false = mean_squared_error(y_cmntf, cat_results['train'][X_train_cmntf.index], squared=False)
            print(f"f in B  RMSLE={RMSLE_base_false}")
            RMSLE_false   = mean_squared_error(y_cmntf, cat_results_cmntf['train'], squared=False)
            print(f"False   RMSLE={RMSLE_false}")
            RMSLE_merged  = mean_squared_error(y, cat_results_merged['train'], squared=False)
            print(f"merged  RMSLE={RMSLE_merged}")
            print("-"*80)

            if RMSLE_base_false > RMSLE_false:
                results = cat_results_merged
            else:
                results = cat_results

            y_pred = pd.Series(results['train']).copy()
            y_pred.where((y_pred<14)&(X_train.comments_disabled==True), y_pred*0.99965, inplace=True)
            y_pred.where((y_pred>14)&(X_train.comments_disabled==True), y_pred*1.0001, inplace=True)
            y_pred.where((y_pred<14)&(X_train.ratings_disabled==True), y_pred*0.99965, inplace=True)
            y_pred.where((y_pred>14)&(X_train.ratings_disabled==True), y_pred*1.0001, inplace=True)

            RMSLE_merged  = mean_squared_error(y, y_pred, squared=False)
            print(f"merged2 RMSLE={RMSLE_merged}")
            print("-"*80)

            sunmission = output_results(results, test_id, seed, N_FOLDS, dataid, out_dir)

            #-----------------------------------------------
            # CATBOOST MODEL 2
            #-----------------------------------------------
            # df[df['comments_disabled']==False]の学習、予測
            cat_params = {'bootstrap_type': 'Bayesian', 
                          'boosting_type': 'Plain', #'Ordered', #'Plain',
                          'iterations':50000,
                          'depth': 16,
                          'loss_function': 'RMSE',
                          'eval_metric': 'RMSE',
                          'learning_rate': 1e-2,
                          'leaf_estimation_method': 'Gradient', #'Newton', 'Exact'
                          'l2_leaf_reg': 1.0,
                          'random_strength': 0.8,
                          'bagging_temperature': 0.9,
                          'has_time': False,
                          'grow_policy': 'Lossguide', # 'SymmetricTree', #'Depthwise'
                          'min_data_in_leaf': 1,
                          'max_leaves': 31,
                          'random_seed': seed,
                          #'one_hot_max_size': len(cat_features),
                          'task_type': 'GPU'}
            if SKF:
                cat_results_cmntf = run_cv_model(train=X_train_cmntf, test=X_test_cmntf, target=y_cmntf, target_skf=y_bin_cmntf, encoding=[],
                                                 model_fn=runCAT, params=cat_params, eval_fn=rmsle, label='CatBoostRegressor2',
                                                 cv=N_FOLDS, repeats=1, seed=seed)
                # df全体の学習、予測
                cat_results = run_cv_model(train=X_train, test=X_test, target=y, target_skf=y_bin, encoding=[],
                                           model_fn=runCAT, params=cat_params, eval_fn=rmsle, label='CatBoostRegressor2',
                                           cv=N_FOLDS, repeats=1, seed=seed)
            else:
                cat_results_cmntf = run_cv_model(train=X_train_cmntf, test=X_test_cmntf, target=y_cmntf, target_skf=None, encoding=[],
                                                 model_fn=runCAT, params=cat_params, eval_fn=rmsle, label='CatBoostRegressor2',
                                                 cv=N_FOLDS, repeats=1, seed=seed)
                # df全体の学習、予測
                cat_results = run_cv_model(train=X_train, test=X_test, target=y, target_skf=None, encoding=[],
                                           model_fn=runCAT, params=cat_params, eval_fn=rmsle, label='CatBoostRegressor2',
                                           cv=N_FOLDS, repeats=1, seed=seed)

            cat_results_merged = {
                "label": "CatBoostRegressor2",
                "train": pd.concat([pd.Series(cat_results_cmntf['train'], index=X_train_cmntf.index),
                                    pd.Series(cat_results['train'][X_train_rest.index], index=X_train_rest.index)]).sort_index(),
                "test":  pd.concat([pd.Series(cat_results_cmntf['test'], index=X_test_cmntf.index),
                                    pd.Series(cat_results['test'][X_test_rest.index], index=X_test_rest.index)]).sort_index()}

            print("-"*80)
            RMSLE_base    = mean_squared_error(y, cat_results['train'], squared=False)
            print(f"Base    RMSLE={RMSLE_base}")
            RMSLE_base_false = mean_squared_error(y_cmntf, cat_results['train'][X_train_cmntf.index], squared=False)
            print(f"f in B  RMSLE={RMSLE_base_false}")
            RMSLE_false   = mean_squared_error(y_cmntf, cat_results_cmntf['train'], squared=False)
            print(f"False   RMSLE={RMSLE_false}")
            RMSLE_merged  = mean_squared_error(y, cat_results_merged['train'], squared=False)
            print(f"merged  RMSLE={RMSLE_merged}")
            print("-"*80)

            if RMSLE_base_false > RMSLE_false:
                results = cat_results_merged
            else:
                results = cat_results

            sunmission = output_results(results, test_id, seed, N_FOLDS, dataid, out_dir)


In [ ]:
print(datetime.datetime.now()-start)